# Jupyter notebook test

I am starting on working through the preprocessing workflow. I think I agree with Dr. Vb, the goal will be to try to create a "density plot" along a lane (similar to what it seems like was done in: Chabrun et al. 2023). Then normalize it so that all lanes are aligned and then train models off of that.



In [1]:
# pip install -r Hu_etal2023_IFE_AI-master/requirements.txt


In [2]:
# ---------------------
# ---- Check if imports are installed -> if Not install them ------
# ---------------------
import importlib
import subprocess
import sys

def install_if_missing(package):
    """Check if a package is installed, and install it if not."""
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"📦 Installing missing package: {package}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for pkg in ["matplotlib", "numpy", "pandas", 
    "opencv-python", "Pillow", "PYMIC", "torch", 
    "scikit-image", "scikit-learn",
    "tensorboard", "torchvision"]:
    
    install_if_missing(pkg)

📦 Installing missing package: matplotlib
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 9.6 MB/s  0:00:00m 9.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.6 MB/s  0:00:001.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.3 MB/s  0:00:00 12.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 11.3 MB/s  0:00:002.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [matplotlib] 7/8 [matplotlib]



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


📦 Installing missing package: pandas
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 9.2 MB/s  0:00:01.1 MB/s eta 0:00:01m01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]━━━━━━━━━━━ 2/3 [pandas]
📦 Installing missing package: opencv-python



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 9.2 MB/s  0:00:049.4 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 8.6 MB/s  0:00:00m 7.8 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]m 1/2 [opencv-python]
📦 Installing missing package: Pillow


...


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


📦 Installing missing package: scikit-learn
📦 Installing missing package: torchvision



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.6 MB/s  0:00:00



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Set working directories and set directory for where files are located

In [4]:
import os
import pandas as pd
import numpy as np
import cv2


# projectdirect = "C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/"
projectdirect = "/Users/cvieni/Documents/Path_alpha1Gels_VB"
working_direct = projectdirect
# working_direct = os.path.join(projectdirect, "Wilrich_alpha1_Isofocus_machinelearning/")
# C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/Wilrich_alpha1_Isofocus_machinelearning/gel_scans

img_direct = os.path.join(working_direct, "gel_scans/")

test_image = "25331.jpg"  # Change this to your PDF file
test_img_path = os.path.join(img_direct, test_image)


test_image = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)

preprocess_gel(test_image)

NameError: name 'preprocess_gel' is not defined

# First: Generate file list & cleanup input image

In [ ]:
# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
TRACE_LENGTH = 512
EXPECTED_LANES = 18
LANE_MIN_WIDTH_PX = 20
S_WINDOW = (0.40, 0.47)
Z_WINDOW = (0.55, 0.62)

# ---------------------------
# 1. LOAD & PREPROCESS GEL IMAGE
# ---------------------------
def preprocess_gel(img_path):
    img = cv2.imread(img_path) # read array of image
    if img.ndim == 3: # 3 = color image; dimensions = 2 = grayscale
        g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        g = img.copy()
    
    g = cv2.bitwise_not(g)  # make bands bright
    
    # background removal using morphological opening
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (51,51))
    bg = cv2.morphologyEx(g, cv2.MORPH_OPEN, kernel)
    
    g_proc = cv2.subtract(g, bg)
    g_proc = cv2.medianBlur(g_proc, 3)
    
    return g_proc, bg  # return both processed image and background


# -------------------------------
# Load & preprocess
# -------------------------------
preprocessed_image, background = preprocess_gel(test_img_path)

# Optional: resize for display
scale_percent = 50
def resize_img(img, scale):
    width = int(img.shape[1] * scale / 100)
    height = int(img.shape[0] * scale / 100)
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

resized_preprocessed = resize_img(preprocessed_image, scale_percent)
resized_bg = resize_img(background, scale_percent)

# -------------------------------
# Visualize using OpenCV
# -------------------------------
cv2.imshow("Preprocessed Gel", resized_preprocessed)
cv2.imshow("Background", resized_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:

test_image = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)
img_invert = cv2.bitwise_not(test_image)


# ---------------------------
# 1. LOAD & PREPROCESS GEL IMAGE
# ---------------------------
def preprocess_gel(img_path):
    img = cv2.imread(img_path) # read array of image
    if img.ndim == 3: # 3 = color image; dimensions = 2 = grayscale
        g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        g = img.copy()
    
    g = cv2.bitwise_not(g)  # make bands bright
    # background removal using morphological opening
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (51,51))
    bg = cv2.morphologyEx(g, cv2.MORPH_OPEN, kernel)
    g = cv2.subtract(g, bg)
    g = cv2.medianBlur(g, 3) # smooth image with Gaussian blur
    return g

# ---------------------------
# 2. LANE DETECTION
# ---------------------------
def detect_lanes(img, expected_n=EXPECTED_LANES, min_sep=LANE_MIN_WIDTH_PX):
    # Take 2D image and measure sum along the vertical row to create a 1D array of 
    # length width w/ the total intensity of all pixels in that column
    # if 3D (color image) then vx = sum of RGB values in each column
    vx = img.sum(axis=0).astype(np.float32) # axis = 0 -> sum along the vertical axis
    
    # normalize all values to center around 0 -> add 1e-6 in case Std is "0"
    # should improve peak detection as the band threshold becomes relative vs. intensity
    vx = (vx - vx.mean()) / (vx.std() + 1e-6)
    
    # find_peaks = SciPy function to take 1d array and find local maxima
    # "_" is a dictionary that we don't need here
    peaks, _ = find_peaks(vx, distance=min_sep)
    
    # pick top expected_n peaks
    if len(peaks) > expected_n:
        peaks = peaks[np.argsort(vx[peaks])[-expected_n:]]
    peaks = np.sort(peaks)
    
    lanes = []
    half_width = min_sep // 2
    for p in peaks:
        x0 = max(p - half_width, 0)
        x1 = min(p + half_width, img.shape[1]-1)
        lanes.append((x0, x1))
    return lanes

# ---------------------------
# 3. EXTRACT 1D TRACE
# ---------------------------
def lane_trace(img, x0, x1, length=TRACE_LENGTH):
    roi = img[:, x0:x1]
    trace = roi.mean(axis=1)  # collapse to 1D
    # normalize
    trace = (trace - np.percentile(trace, 5)) / (np.percentile(trace, 95) - np.percentile(trace, 5) + 1e-6)
    # resample
    trace_resampled = np.interp(np.linspace(0, len(trace)-1, length), np.arange(len(trace)), trace)
    return trace_resampled


img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Optional: invert image if bands are dark
img = cv2.bitwise_not(img)

# ---------------------------
# DEFINE LANE (column range)
# ---------------------------
x0, x1 = 100, 130  # example column indices of a lane

# ---------------------------
# COMPUTE 1D TRACE
# ---------------------------
trace = lane_trace(img, x0, x1)

# ---------------------------
# OVERLAY TRACE ON IMAGE
# ---------------------------
plt.figure(figsize=(8,10))
plt.imshow(img, cmap='gray', aspect='auto')

# Scale the normalized trace to the width of the lane
lane_center = (x0 + x1) / 2
lane_width = x1 - x0
# scale trace to lane width (pixels)
trace_scaled = trace * lane_width
# x-coordinates of the trace: center + trace offset
x_overlay = lane_center - lane_width/2 + trace_scaled
# y-coordinates: linearly from top to bottom
y_overlay = np.linspace(0, img.shape[0]-1, TRACE_LENGTH)

plt.plot(x_overlay, y_overlay, color='red', linewidth=2, label='Lane intensity trace')
plt.title(f"Lane intensity overlay: columns {x0}-{x1}")
plt.xlabel("Horizontal pixels")
plt.ylabel("Vertical pixels")
plt.legend()
plt.gca().invert_yaxis()  # optional: top of gel = 0
plt.show()



NameError: name 'img' is not defined

In [12]:
def remove_small_components(img): # 2D or 3D
    if(img.sum()==0):
        print('the largest component is null')
        return img
    if(len(img.shape) == 3):
        s = ndimage.generate_binary_structure(3,1) # iterate structure
    elif(len(img.shape) == 2):
        s = ndimage.generate_binary_structure(2,1) # iterate structure
    else:
        raise ValueError("the dimension number shoud be 2 or 3")

    labeled_array, numpatches = ndimage.label(img,s) # labeling
    label = np.zeros_like(img)
    for i in range(numpatches):
        temp_label = labeled_array == (i + 1)
        if(temp_label.sum() > 10):
            label = np.maximum(label, temp_label)
    return np.asarray(label, np.uint8)


remove_small_components(test_image)


AttributeError: 'str' object has no attribute 'sum'

In [ ]:




for filename in os.listdir(img_direct):
    if filename.lower().endswith(".jpg"):
        
        # Run script for file given by filename
        print("Running image cleanup & registration for:", filename)
        
        # file_path = os.path.join(img_direct, filename)
        # print("Directory:", os.path.dirname(file_path))


Running image cleanup & registration for: 25331.jpg
Directory: C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/Wilrich_alpha1_Isofocus_machinelearning/gel_scans
Running image cleanup & registration for: 25332.jpg
Directory: C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/Wilrich_alpha1_Isofocus_machinelearning/gel_scans
Running image cleanup & registration for: 25333.jpg
Directory: C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/Wilrich_alpha1_Isofocus_machinelearning/gel_scans
Running image cleanup & registration for: 25334.jpg
Directory: C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/Wilrich_alpha1_Isofocus_machinelearning/gel_scans
Running image cleanup & registration for: 25335.jpg
Directory: C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/Wilrich_alpha1_Isofocus_machinelearning/gel_scans
Running image cleanup & registration for: 25336.jpg
Directory: C:/Users/M297055/OneDrive - Mayo Clinic/Documents/Research/Wilrich_alpha1_Iso

In [ ]:
# Reference code from Hu et al. ---> will take peace by peace to implement

In [ ]:
def remove_small_components(img): # 2D or 3D
    if(img.sum()==0):
        print('the largest component is null')
        return img
    if(len(img.shape) == 3):
        s = ndimage.generate_binary_structure(3,1) # iterate structure
    elif(len(img.shape) == 2):
        s = ndimage.generate_binary_structure(2,1) # iterate structure
    else:
        raise ValueError("the dimension number shoud be 2 or 3")

    labeled_array, numpatches = ndimage.label(img,s) # labeling
    label = np.zeros_like(img)
    for i in range(numpatches):
        temp_label = labeled_array == (i + 1)
        if(temp_label.sum() > 10):
            label = np.maximum(label, temp_label)
    return np.asarray(label, np.uint8)

def get_color_region(Im):
    """
    get the region of bands
    """
    Im = np.asarray(Im, np.float32)
    Ir = Im[:, :, 0] 
    Ig = Im[:, :, 1]
    Ib = Im[:, :, 2]
    diff = (Ir - Ib) * (Ir - Ib) + (Ig - Ib) * (Ig - Ib)
    mask = (diff > 200) * (Ib > 200)
    return mask

def remove_background(Im, b_color):
    """
    remove noise in the background of each lane,
    and set the intesnity values to b_color
    """
    Ir = Im[:, :, 0] 
    Ig = Im[:, :, 1]
    Ib = Im[:, :, 2]
    H, W = Ir.shape
    mask = get_color_region(Im)
    mask = ndimage.morphology.binary_closing(mask)
    mask = ndimage.morphology.binary_opening(mask)
    if(mask.sum() < 50):
        bg_mask = np.ones_like(mask)
    else:
        mask = remove_small_components(mask)
        indxes = np.nonzero(mask)
        h0 = max(indxes[0].min(), 0)
        h1 = min(indxes[0].max(), H)
        w0 = indxes[1].min()
        w1 = indxes[1].max()
        mask[np.ix_(range(h0, h1), range(w0, w1))] = np.ones((h1 - h0, w1 - w0))
        bg_mask = 1 - mask 

    meanr = b_color[0] * np.ones_like(Ig, np.float32)
    meang = b_color[1] * np.ones_like(Ig, np.float32)
    meanb = b_color[2] * np.ones_like(Ib, np.float32)
    Ir_out, Ig_out, Ib_out = Ir * 1, Ig * 1, Ib * 1 
    Ir_out[bg_mask > 0] = meanr[bg_mask > 0]
    Ig_out[bg_mask > 0] = meang[bg_mask > 0]
    Ib_out[bg_mask > 0] = meanb[bg_mask > 0]
    I1 = np.asarray([Ir_out, Ig_out, Ib_out])
    I1 = np.transpose(I1, [1, 2, 0])
    I1 = np.asarray(I1, np.uint8)
    return I1 

def  remove_margin_and_background(Im, w=30, m = 8):
    """
    remove margin and background for images from new system
    """
    shape = Im.shape
    Ir = Im[:, :, 0] 
    Ig = Im[:, :, 1]
    Ib = Im[:, :, 2]
    Ir_m = np.percentile(Ir, 50)
    Ig_m = np.percentile(Ig, 50)
    Ib_m = np.percentile(Ib, 50)

    H, W = shape[0], shape[1]
    Isub_list = []
    for i in range(6):
        Isub = Im[:, (w + m )*i : (w + m)*i + w, :]
        Isub = remove_background(Isub, [Ir_m, Ig_m, Ib_m])
        Isub_list.append(Isub)
    Isub = np.concatenate(Isub_list, axis = 1)
    return Isub
    
def preprocess_of_one_image(I, group = 'a'):
    """
    preprocess of images for new system
    step 1 crop the lanes with a margin
    step 2 for each lane, get the background region and replace 
           the color as their average to reduce noise
    step 3 resize the image in to 144 (W) x 144 (H)
    group: 'a' -- for images in the new system
           'b' -- for images in the old system
    """
    Im = np.asarray(I)
    [W, H] = I.size
    if(group == 'a'):
        assert(W == 240 and H == 210)
        Isub = Im[63:173, 10:230, :]   # output size 110 x 220
        Isub = remove_margin_and_background(Isub) # output size 110 x 180
    else:
        Isub = Im[35:, 7:-7, :] 
        Isub = remove_margin_and_background(Isub, w=50, m=0)
    I1 = Image.fromarray(Isub)
    I1 = I1.resize((144, 144),Image.BILINEAR)
    return I1

def preprocess_demo(group = 'a'):
    """
    demo of image preprocessing
    group: 'a' -- for images in the new system
           'b' -- for images in the old system
    """
    in_folder  = "data/data_{0:}".format(group)
    out_folder = in_folder + "_process"
    filenames = os.listdir(in_folder)
    filenames = [item for item in filenames if \
        ".bmp" in item or ".jpg" in item]
    for i in range(len(filenames)):
        print(filenames[i])
        full_name = in_folder + '/' + filenames[i]
        I  = Image.open(full_name)
        I1 = preprocess_of_one_image(I, group)
        plt.subplot(1,2,1); plt.imshow(I); plt.title("Original image")
        plt.subplot(1,2,2); plt.imshow(I1); plt.title("After preprocessing")
        plt.show()
        I1.save(out_folder + '/' + filenames[i])
    
if __name__ == "__main__":
    # group = 'a' for new system, 'b' for old system
    preprocess_demo(group = 'a') 
    preprocess_demo(group = 'b')
    

    